In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import numpy as np
import os
import pandas as pd
import sys
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

2.5.1+cu121
True
NVIDIA GeForce RTX 2070
cuda


In [5]:
# csv_path = '/content/drive/MyDrive/rupa/archive/labeled_anomalies.csv'
csv_path = r'C:\Users\skhandaker\OneDrive - Oklahoma City University\Documents\Anomaly Detection Paper\rupa\archive\labeled_anomalies.csv'

df = pd.read_csv(csv_path)

m_files = [f"{chan_id}.npy" for chan_id in df.loc[df['spacecraft']=='MSL', 'chan_id']]

print(m_files)

['M-6.npy', 'M-1.npy', 'M-2.npy', 'S-2.npy', 'P-10.npy', 'T-4.npy', 'T-5.npy', 'F-7.npy', 'M-3.npy', 'M-4.npy', 'M-5.npy', 'P-15.npy', 'C-1.npy', 'C-2.npy', 'T-12.npy', 'T-13.npy', 'F-4.npy', 'F-5.npy', 'D-14.npy', 'T-9.npy', 'P-14.npy', 'T-8.npy', 'P-11.npy', 'D-15.npy', 'D-16.npy', 'M-7.npy', 'F-8.npy']


In [11]:
# test_dir = '/content/drive/MyDrive/rupa/archive/data/data/test'
test_dir = r'C:\Users\skhandaker\OneDrive - Oklahoma City University\Documents\Anomaly Detection Paper\rupa\archive\data\data\test'

# m_files = sorted(f for f in os.listdir(test_dir) if f.endswith('.npy') and f.startswith('M'))

data_list = [np.load(os.path.join(test_dir, fname)) for fname in m_files]

X = np.concatenate(data_list, axis=0)

print("Concatenated shape (rows):", X.shape)

Concatenated shape (rows): (73729, 55)


In [13]:
print(X)

[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [-1.          0.          0.         ...  0.          0.
   0.        ]
 [-1.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [-1.          0.          0.         ...  0.          0.
   0.        ]
 [-0.86956522  0.          0.         ...  0.          0.
   0.        ]
 [-1.          0.          0.         ...  0.          0.
   0.        ]]


In [9]:
all_rows = []

for f in m_files:
    data = np.load(os.path.join(test_dir, f))
    all_rows.append(data)

combined = np.concatenate(all_rows, axis=0).astype(np.float32)


df = pd.DataFrame(combined, dtype=np.float32)
unique_embeddings = df.drop_duplicates()

print("Unique embeddings:", len(unique_embeddings))
print("Total time steps:", len(df))

Unique embeddings: 25478
Total time steps: 73729


In [15]:
print(unique_embeddings)

             0    1    2    3    4    5    6    7    8    9   ...   45   46  \
0     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3     -1.000000  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
73324  1.043478  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73425  0.260870  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73481 -0.956522  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73564 -0.913043  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73616 -0.913043  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

        47   48   49   50   51   52   53   54  
0  

In [17]:
print(df)

             0    1    2    3    4    5    6    7    8    9   ...   45   46  \
0     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3     -1.000000  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4     -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
73724 -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73725 -0.956522  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73726 -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73727 -0.869565  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
73728 -1.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

        47   48   49   50   51   52   53   54  
0  

In [19]:
GPT_CONFIG = {
  "vocab_size": 25500, # Vocabulary size
  "context_length": 64, # Context length
  "emb_dim": 55, # Embedding dimension
  "n_heads": 5, # Number of attention heads
  "n_layers": 12, # Number of layers
  "drop_rate": 0.1, # Dropout rate
  "qkv_bias": False # Query-Key-Value bias
}

In [21]:
class MultiHeadAttention(nn.Module):
 def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
  super().__init__()
  assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
  self.d_out = d_out
  self.num_heads = num_heads
  self.head_dim = d_out // num_heads
  self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
  self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
  self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
  self.out_proj = nn.Linear(d_out, d_out)
  self.dropout = nn.Dropout(dropout)
  self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
 def forward(self, x):
  b, num_tokens, d_in = x.shape
  keys = self.W_key(x)
  queries = self.W_query(x)
  values = self.W_value(x)
  keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
  values = values.view(b, num_tokens, self.num_heads, self.head_dim)
  queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
  keys = keys.transpose(1, 2)
  queries = queries.transpose(1, 2)
  values = values.transpose(1, 2)
  attn_scores = queries @ keys.transpose(2, 3)
  mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

  attn_scores.masked_fill_(mask_bool, -torch.inf)
  attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
  attn_weights = self.dropout(attn_weights)
  context_vec = (attn_weights @ values).transpose(1, 2)

  context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
  context_vec = self.out_proj(context_vec)
  return context_vec

In [23]:
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

In [25]:
class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), GELU(), nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),)

  def forward(self, x):
    return self.layers(x)

In [27]:
class LayerNorm(nn.Module):
  def __init__(self, emb_dim):
    super().__init__()
    self.eps = 1e-5
    self.scale = nn.Parameter(torch.ones(emb_dim))
    self.shift = nn.Parameter(torch.zeros(emb_dim))
  def forward(self, x):
    mean = x.mean(dim=-1, keepdim=True)
    var = x.var(dim=-1, keepdim=True, unbiased=False)
    norm_x = (x - mean) / torch.sqrt(var + self.eps)
    return self.scale * norm_x + self.shift

In [29]:
class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(d_in=cfg["emb_dim"], d_out=cfg["emb_dim"], context_length=cfg["context_length"], num_heads=cfg["n_heads"], dropout=cfg["drop_rate"], qkv_bias=cfg["qkv_bias"])
    self.ff = FeedForward(cfg)
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate"])
  def forward(self, x):
    shortcut = x
    x = self.norm1(x)
    x = self.att(x)
    x = self.drop_shortcut(x)
    x = x + shortcut
    shortcut = x
    x = self.norm2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut
    return x

In [31]:
c_unique_embeddings = unique_embeddings.values
emb_tuples = [tuple(row) for row in c_unique_embeddings]
emb2idx = {emb: idx for idx, emb in enumerate(emb_tuples)}
def get_embedding_ids(batch):
    batch_np = batch.detach().cpu().numpy().astype(np.float32)

    B, L, D = batch_np.shape
    ids = np.full((B, L), -1, dtype=int)

    for i in range(B):
        for j in range(L):
            ids[i, j] = emb2idx.get(tuple(batch_np[i, j]), -1)

    return ids

In [33]:
class GPTModel(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    # self.cfg = cfg
    self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
    self.drop_emb = nn.Dropout(cfg["drop_rate"])

    self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

    self.final_norm = LayerNorm(cfg["emb_dim"])
    self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

  def forward(self, in_idx):
    global device
    batch_size, seq_len, _ = in_idx.shape
    in_idx = get_embedding_ids(in_idx)

    in_idx = torch.from_numpy(in_idx).long().to(device)

    tok_embeds = self.tok_emb(in_idx)
    pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
    x = tok_embeds + pos_embeds
    x = self.drop_emb(x)
    x = self.trf_blocks(x)
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits

In [35]:
train_ratio = 0.90
split_idx = int(train_ratio * len(X))
train_data = X[:split_idx]
val_data = X[split_idx:]

In [37]:
class GPTDatasetV1(Dataset):
  def __init__(self, txt, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    # for txt in txts:
    for i in range(0, len(txt) - max_length, stride):
      input_chunk = txt[i:i + max_length]
      target_chunk = txt[i + 1: i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [39]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
  dataset = GPTDatasetV1(txt, max_length, stride)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
  return dataloader

In [41]:
train_loader = create_dataloader_v1(
  train_data,
  batch_size=256,
  max_length=GPT_CONFIG["context_length"],
  stride=GPT_CONFIG["context_length"],
  drop_last=True,
  shuffle=True,
  num_workers=0
)

val_loader = create_dataloader_v1(
  val_data,
  batch_size=256,
  max_length=GPT_CONFIG["context_length"],
  stride=GPT_CONFIG["context_length"],
  drop_last=False,
  shuffle=False,
  num_workers=0
)

In [43]:
def calc_loss_batch(input_batch, target_emb_batch, model, device):
    input_batch = input_batch.to(device)
    target_emb_batch = target_emb_batch.to(device)
    logits = model(input_batch)
    target_ids = get_embedding_ids(target_emb_batch)
    target_ids = torch.from_numpy(target_ids).long().to(device)
    B, L, V = logits.size()
    logits_flat  = logits.view(B * L, V)
    targets_flat = target_ids.view(B * L)
    loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
    return loss

In [45]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
  total_loss = 0
  if len(data_loader) == 0:
    return float("nan")
  elif num_batches is None:
    num_batches = len(data_loader)
  else:
    num_batches = min(num_batches, len(data_loader))

  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      total_loss += loss.item()
    else:
      break

  return total_loss / num_batches

In [47]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
  model.eval()
  with torch.no_grad():
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
  model.train()
  return train_loss, val_loss

In [49]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs, eval_freq, eval_iter):
  train_losses, val_losses, track_tokens_seen = [], [], []
  tokens_seen, global_step = 0, -1
  for epoch in range(num_epochs):
    model.train()
    for input_batch, target_batch in train_loader:
      optimizer.zero_grad()
      loss = calc_loss_batch(input_batch, target_batch, model, device)
      loss.backward()
      optimizer.step()
      tokens_seen += input_batch.numel()
      global_step += 1

      if global_step % eval_freq == 0:
        train_loss, val_loss = evaluate_model(model, train_loader, val_loader, device, eval_iter)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        track_tokens_seen.append(tokens_seen)
        print(f"Ep {epoch+1} (Step {global_step:06d}): Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
  return train_losses, val_losses, track_tokens_seen

In [51]:
torch.manual_seed(42)
model = GPTModel(GPT_CONFIG)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(
  model.parameters(),
  lr=0.0004, weight_decay=0.1
)
num_epochs = 1
train_losses, val_losses, tokens_seen = train_model_simple(
  model, train_loader, val_loader, optimizer, device,
  num_epochs=num_epochs, eval_freq=5, eval_iter=1
)

Ep 1 (Step 000000): Train loss 10.279, Val loss 10.181


In [81]:
context = GPT_CONFIG["context_length"]
forecast_horizon = context

model.eval()

with torch.no_grad():
    for t in range(len(val_data) - context - forecast_horizon):
        init_seq = val_data[t : t+context]
        init_seq = init_seq[np.newaxis]
        init_seq = torch.tensor(init_seq, device=device)
        true_seq = val_data[t+context : t+context+forecast_horizon]

        # print(pred_emb)
        predicted_embs = []
        for _ in range(forecast_horizon):
            logits = model(init_seq)
            next_id = logits[0, -1].argmax().item()
            pred_emb = emb_tuples[next_id]
            predicted_embs.append(pred_emb)
        # print(predicted_embs)
        # print(true_seq)
        dist = np.linalg.norm(predicted_embs - true_seq)
        print(dist)
            


14.247915272635977
7.98386254184753
8.656336474587334
7.98886252763068
9.023997517736388
7.9750229206168655
16.514930167676898
18.385314779849054
14.545465816013378
9.38739325036704
18.368198044579138
7.974583962582878
16.482078549387225
18.349080441232072
16.469640194422478
7.910358993124892
7.901534467248158
11.247022521741265
7.731475566131577
7.308727171859933
9.051127858063138
7.181341335591885
6.6530181219617575
14.447353068641473
6.584729090870785
6.042838385452925
16.295238730671368
6.036860324389807
7.432046688129004
14.341121710156694
9.995226306048066
12.294837781310628
6.192305225701958
14.464475654862516
11.570015464702
15.180641138247406
9.218631254207711
11.067600412404124
11.751574571617299
14.254624330144091
8.00738048280576
17.322060050525973
17.062386470696577
6.079094311430087
6.38252465711247
14.68425207520717
10.860253993230389
15.429088228469729
14.589496794282729
14.548395814370437
16.5712313391209
15.469748308063071
7.017635114676915
6.624335811615548
6.3389468

KeyboardInterrupt: 

In [97]:
for m in model.modules():
    if isinstance(m, nn.Dropout):
        m.train()

with torch.no_grad():
    for t in range(1):
        init_seq = val_data[t : t+context]
        init_seq = init_seq[np.newaxis]
        init_seq = torch.tensor(init_seq, device=device)
        true_seq = val_data[t+context : t+context+forecast_horizon]

        # print(pred_emb)
        predicted_embs = []
        for _ in range(2):
            logits = model(init_seq)
            print(logits[0, -1])
            predicted_embs.append(logits[0, -1])
        
        predicted_embs = torch.cat(predicted_embs, 0)
        # print(predicted_embs)
        mean = predicted_embs.mean(0)
        # print(mean)

tensor([-0.1561, -0.3602,  0.2979,  ...,  0.3252,  0.2796, -1.4058],
       device='cuda:0')
tensor([-0.3346, -0.2107,  1.0144,  ...,  0.8399,  0.7852, -0.8823],
       device='cuda:0')
